In [1]:
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import json
import os
import time

In [2]:
load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


In [3]:
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)

In [4]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        db_name = result.fetchone()[0]
        print(f"Connected to database: {db_name}")
except Exception as e:
    print(f"Error: {e}")

Connected to database: itt_master_contents


### Test with `GiDestinationId`

In [5]:
from sqlalchemy import create_engine, Table, MetaData, text
from sqlalchemy.orm import sessionmaker
import json

# Database setup
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)

# Reflect tables
metadata = MetaData()
metadata.reflect(bind=engine)
hotel_info_all = Table('hotel_info_all', metadata, autoload_with=engine)
innova_hotels_main = Table('innova_hotels_main', metadata, autoload_with=engine)

# Define JSON mapping from 'hotel_info_all' JSON content to 'innova_hotels_main'
json_mapping = {
    'HotelName': 'name',
    'HotelId': 'systemId',
    'AddressLine1': 'address.line1',
    'AddressLine2': 'address.line2',
    'State': 'address.stateName',
    'StateCode': 'address.stateCode',
    'PostCode': 'address.zipCode',
    'City': 'address.cityName',
    'CityCode': 'address.CityCode',
    'Country': 'address.countryName',
    'CountryCode': 'address.countryCode',
    'Latitude': 'geocode.lat',
    'Longitude': 'geocode.lon',
    'ContactNumber': 'contact.phoneNo',
    'Website': 'contact.website',
    'HotelStar': 'rating',
    'PrimaryPhoto': 'imageUrls',  
    'HotelReview': 'tripAdvisorRating',
    'DestinationId': 'giDestinationId',
    'Amenities_1': 'masterRoomAmenities',
    'Amenities_2': 'masterHotelAmenities',
    'Amenities_3': 'masterHotelAmenities',
    'Amenities_4': 'masterHotelAmenities',
    'Amenities_5': 'masterHotelAmenities'
}

# Start a session and use the 'begin()' method to ensure data is saved immediately
session = Session()

# Function to escape single quotes in string values
def escape_single_quotes(value):
    if isinstance(value, str):
        return value.replace("'", "''")  # Escape single quotes
    return value

try:
    # Begin a transaction
    with session.begin():
        # Query for 'GiDestinationId' where 'StatusUpdateHotelInfo' is 'Done Json'
        results = (
            session.query(hotel_info_all)
            .filter(hotel_info_all.c.StatusUpdateHotelInfo == 'Done Json')
            .group_by(hotel_info_all.c.GiDestinationId)  
            .all()
        )

        for row in results:
            # Assuming 'HotelInfo' is already a dictionary, no need to parse with json.loads
            hotel_info = row.HotelInfo
            data = {}

            # Map JSON content to 'innova_hotels_main' columns
            for target_col, json_key in json_mapping.items():
                if target_col == 'PrimaryPhoto':  # Special handling for the first image URL
                    # Check if imageUrls is not empty
                    image_urls = hotel_info.get('imageUrls', [])
                    if image_urls:
                        data['PrimaryPhoto'] = image_urls[0]  
                    else:
                        data['PrimaryPhoto'] = None  
                elif target_col.startswith('Amenities_'): 
                    amenities_key = json_mapping[target_col]
                    amenities = hotel_info.get(amenities_key, [])
                    data[target_col] = ', '.join(amenities) if amenities else None  
                else:
                    # Handle normal mappings, including nested JSON keys
                    keys = json_key.split('.') 
                    value = hotel_info
                    for key in keys:
                        value = value.get(key) if value else None  # Traverse JSON, set None if key not found
                    data[target_col] = value if value is not None else None  # D

            data['SupplierCode'] = 'oryx'
            # Escape all string values to prevent SQL injection or syntax errors
            data = {k: escape_single_quotes(v) for k, v in data.items()}

            # Set missing columns to NULL
            for col in innova_hotels_main.columns:
                if col.name not in data:
                    data[col.name] = None

            # Use raw SQL for ON DUPLICATE KEY UPDATE
            # Prepare the insert SQL statement with ON DUPLICATE KEY UPDATE
            columns = ', '.join(data.keys())  # Column names
            values = ', '.join([f"'{v}'" if v is not None else 'NULL' for v in data.values()])  # Values
            update_clause = ', '.join([f"{col} = VALUES({col})" for col in data.keys()])  # Update values on conflict
            
            # Construct the full SQL statement
            sql = f"""
                INSERT INTO innova_hotels_main ({columns}) 
                VALUES ({values}) 
                ON DUPLICATE KEY UPDATE {update_clause}
            """
            
            # Execute the raw SQL query using sqlalchemy.text()
            session.execute(text(sql))

    # Commit transaction (handled by the 'begin()' context)
    print("Data updated successfully in innova_hotels_main.")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()


Data updated successfully in innova_hotels_main.


### Main

In [6]:
from sqlalchemy import create_engine, Table, MetaData, text
from sqlalchemy.orm import sessionmaker
import json

# Database setup
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)

# Reflect tables
metadata = MetaData()
metadata.reflect(bind=engine)
hotel_info_all = Table('hotel_info_all', metadata, autoload_with=engine)
innova_hotels_main = Table('innova_hotels_main', metadata, autoload_with=engine)

# Define JSON mapping from 'hotel_info_all' JSON content to 'innova_hotels_main'
json_mapping = {
    'HotelName': 'name',
    'HotelId': 'systemId',
    'AddressLine1': 'address.line1',
    'AddressLine2': 'address.line2',
    'State': 'address.stateName',
    'StateCode': 'address.stateCode',
    'PostCode': 'address.zipCode',
    'City': 'address.cityName',
    'CityCode': 'address.CityCode',
    'Country': 'address.countryName',
    'CountryCode': 'address.countryCode',
    'Latitude': 'geocode.lat',
    'Longitude': 'geocode.lon',
    'ContactNumber': 'contact.phoneNo',
    'Website': 'contact.website',
    'HotelStar': 'rating',
    'PrimaryPhoto': 'imageUrls',  
    'HotelReview': 'tripAdvisorRating',
    'DestinationId': 'giDestinationId',
    'Amenities_1': 'masterRoomAmenities',
    'Amenities_2': 'masterHotelAmenities'
}

# Start a session and use the 'begin()' method to ensure data is saved immediately
session = Session()

# Function to escape single quotes in string values
def escape_single_quotes(value):
    if isinstance(value, str):
        return value.replace("'", "''")  
    return value

try:
    # Begin a transaction
    with session.begin():
        # Query for 'GiDestinationId' where 'StatusUpdateHotelInfo' is 'Done Json'
        results = (
            session.query(hotel_info_all)
            .filter(hotel_info_all.c.StatusUpdateHotelInfo == 'Done Json')
            .group_by(hotel_info_all.c.SystemId)  
            .all()
        )

        for row in results:
            # Assuming 'HotelInfo' is already a dictionary, no need to parse with json.loads
            hotel_info = row.HotelInfo
            data = {}

            # Map JSON content to 'innova_hotels_main' columns
            for target_col, json_key in json_mapping.items():
                if target_col == 'PrimaryPhoto': 
                    # Check if imageUrls is not empty
                    image_urls = hotel_info.get('imageUrls', [])
                    if image_urls:
                        data['PrimaryPhoto'] = image_urls[0]  
                    else:
                        data['PrimaryPhoto'] = None  
                elif target_col.startswith('Amenities_'): 
                    amenities_key = json_mapping[target_col]
                    amenities = hotel_info.get(amenities_key, [])
                    data[target_col] = ', '.join(amenities) if amenities else None  
                else:
                    # Handle normal mappings, including nested JSON keys
                    keys = json_key.split('.') 
                    value = hotel_info
                    for key in keys:
                        value = value.get(key) if value else None 
                    data[target_col] = value if value is not None else None  

            
            data['SupplierCode'] = 'oryx'
            # Escape all string values to prevent SQL injection or syntax errors
            data = {k: escape_single_quotes(v) for k, v in data.items()}

            # Set missing columns to NULL
            for col in innova_hotels_main.columns:
                if col.name not in data:
                    data[col.name] = None

            # Use raw SQL for ON DUPLICATE KEY UPDATE
            columns = ', '.join(data.keys())  # Column names
            values = ', '.join([f"'{v}'" if v is not None else 'NULL' for v in data.values()])  
            update_clause = ', '.join([f"{col} = VALUES({col})" for col in data.keys()]) 
            
            # Construct the full SQL statement
            sql = f"""
                INSERT INTO innova_hotels_main ({columns}) 
                VALUES ({values}) 
                ON DUPLICATE KEY UPDATE {update_clause}
            """
            
            # Execute the raw SQL query using sqlalchemy.text()
            session.execute(text(sql))

    # Commit transaction (handled by the 'begin()' context)
    print("Data updated successfully in innova_hotels_main.")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()


Data updated successfully in innova_hotels_main.


### Try to handel with auto commit.

In [24]:
from sqlalchemy import create_engine, Table, MetaData, text
from sqlalchemy.orm import sessionmaker
import json

# Database setup
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)

# Reflect tables
metadata = MetaData()
metadata.reflect(bind=engine)
hotel_info_all = Table('hotel_info_all', metadata, autoload_with=engine)
innova_hotels_main = Table('innova_hotels_main', metadata, autoload_with=engine)

# Define JSON mapping from 'hotel_info_all' JSON content to 'innova_hotels_main'
json_mapping = {
    'HotelName': 'name',
    'HotelId': 'systemId',
    'AddressLine1': 'address.line1',
    'AddressLine2': 'address.line2',
    'State': 'address.stateName',
    'StateCode': 'address.stateCode',
    'PostCode': 'address.zipCode',
    'City': 'address.cityName',
    'CityCode': 'address.CityCode',
    'Country': 'address.countryName',
    'CountryCode': 'address.countryCode',
    'Latitude': 'geocode.lat',
    'Longitude': 'geocode.lon',
    'ContactNumber': 'contact.phoneNo',
    'Website': 'contact.website',
    'HotelStar': 'rating',
    'PrimaryPhoto': 'imageUrls',  
    'HotelReview': 'tripAdvisorRating',
    'DestinationId': 'giDestinationId',
    'Amenities_1': 'masterRoomAmenities',
    'Amenities_2': 'masterHotelAmenities'
}


# Start a session and use the 'begin()' method to ensure data is saved immediately
session = Session()

# Function to escape single quotes in string values
def escape_single_quotes(value):
    if isinstance(value, str):
        return value.replace("'", "''")  
    return value

try:
    # Begin a transaction
    with session.begin():
        results = (
            session.query(hotel_info_all)
            .filter(hotel_info_all.c.StatusUpdateHotelInfo == 'Done Json')
            .group_by(hotel_info_all.c.GiDestinationId)  
            .all()
        )

        for row in results:
            hotel_info = row.HotelInfo
            data = {}

            # Map JSON content to 'innova_hotels_main' columns
            for target_col, json_key in json_mapping.items():
                if target_col == 'PrimaryPhoto': 
                    image_urls = hotel_info.get('imageUrls', [])
                    if image_urls:
                        data['PrimaryPhoto'] = image_urls[0]  
                    else:
                        data['PrimaryPhoto'] = None  
                elif target_col.startswith('Amenities_'): 
                    amenities_key = json_mapping[target_col]
                    amenities = hotel_info.get(amenities_key, [])
                    data[target_col] = ', '.join(amenities) if amenities else None  
                else:
                    keys = json_key.split('.') 
                    value = hotel_info
                    for key in keys:
                        value = value.get(key) if value else None  
                    data[target_col] = value if value is not None else None 

            data['SupplierCode'] = 'oryx'
            data = {k: escape_single_quotes(v) for k, v in data.items()}

            # Set missing columns to NULL
            for col in innova_hotels_main.columns:
                if col.name not in data:
                    data[col.name] = None

            # Prepare the insert SQL statement with ON DUPLICATE KEY UPDATE
            columns = ', '.join(data.keys())  
            values = ', '.join([f"'{v}'" if v is not None else 'NULL' for v in data.values()])  
            update_clause = ', '.join([f"{col} = VALUES({col})" for col in data.keys()])  
            
            # Construct the full SQL statement
            sql = f"""
                INSERT INTO innova_hotels_main ({columns}) 
                VALUES ({values}) 
                ON DUPLICATE KEY UPDATE {update_clause}
            """
            
            session.execute(text(sql))

    # Commit the transaction after all operations in the context manager
    session.commit()

    print("Data updated successfully in innova_hotels_main.")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()


Data updated successfully in innova_hotels_main.
